# semantic grep vectorspace test bench

## pending design decision 
index or ctx or ass vectors?

## theory

five levels:

all presuppose existence of reference space with ctx vectors and ass space with other ctx vectors. should those be made with lemma or surface form?

should reference space addition be weighted not by pmi or idf but by referenitality of the concept?


1. associative-discourse level: wide context window; invidivual vs class?
2. reference: instance vs class, which?; narrow context window; not all words relevant; invidivual vs class?
3. syntax+semantics (roles, case, adverbials): use lemma or surface form?
4. morph (number, definite, tense): use index vector, use lemma or surface form?
5. situ (ĺatlong time source audience channel)

clause adverbials
negation
subordination
coordination


## what to test with?

- ingest fdg and use it for first tests
- ingest propbank 


In [19]:
import re
import math
import random
import time
import nltk

In [6]:
wordspacefile="/home/jussi/Desktop/data/wordspaces/gavagai-news-22-morethan5.wordspace"
dimensionality=2000
denseness=10
fdgtextfile="/home/jussi/Desktop/data/GH95-fdg/950102.sgml.fdg.xml"
#fdgtextfile="/home/jussi/Desktop/data/mini.fdg"

In [11]:
def dofdgfile(fdgfilename):
    utterances=[]
    gameon=False
    taglinepattern = re.compile("^<([\/\w]+)>\s*$")
    sgmllinepattern = re.compile("^\d+\s+<(\w+)>\s+<\w+>\s*$")
    wordlinepattern = re.compile("^(\d+)\s+([\d\/\'\w]+)\s+([\d\/\w]+)\s+(\w+):>(\d+)\s+(.*)$")
    fdgfile=open(fdgfilename,'r')
    a=0
    b=0
    u=0
    w=0

    fdgline=fdgfile.readline()
    notes={}
    tokens=[]
    itemindex={}
    utterance=clause("empty")
    agent=None
    patient=None
    verb=None
    surfacestring=""
    while fdgline:
        a+=1
        tag=taglinepattern.match(fdgline)
        eos=sgmllinepattern.match(fdgline)
        word=wordlinepattern.match(fdgline)
        if tag:
#            print(">",m.groups()[0],"<")
            if (tag.groups()[0] == "HEADLINE"  or tag.groups()[0] == "TEXT" ):
                gameon=True
            if (tag.groups()[0] == "/HEADLINE" or tag.groups()[0] == "/TEXT" ):
                gameon=False
        elif eos:
            u+=1
#            if (len(tokens) > 1):
#                print(tokens)

#            if not utterance is None:
            if len(tokens) > 0:
                utterance.surfacestring=surfacestring 
                utterance.agent=agent
                utterance.patient=patient
                utterance.verb=verb
                utterance.tokens=tokens
                utterances.append(utterance)
            utterance=clause("empty")
            tokens=[]
            itemindex={}
            notes={}
            agent=None
            patient=None
            verb=None
            surfacestring=""

        elif word:
            if gameon:
                w+=1
                note=""
                lemma=word.groups()[2]
                surface=word.groups()[1]
                if (lemma in indexspace):
                    pass
                else:
                    indexspace[lemma] = newrandomvector(dimensionality,denseness)
                    vectorspace[lemma] = {}
                if (surface in indexspace):
                    pass
                else:
                    indexspace[surface] = newrandomvector(dimensionality,denseness)
                    vectorspace[surface] = {}
                position=word.groups()[0]
                role=word.groups()[3]
                head=word.groups()[4]
                morph=word.groups()[5]
                itemindex[position] = lemma
                if position in notes:
                    note=notes[position]
                if role=="det":
                    if surface=="a" or surface=="an":
                        notes[head]="INDEFINITE"
                    else:
                        notes[head]="DEFINITE"
                elif role=="main":
                    verb=event(lemma)
                    tokens.append(lemma)
                    utterance.event = verb
                    utterance.event.tense=="PRESENT"
                elif role=="subj":
                    notes[position]="AGENT" # for now
                    agent=referent(surface)
                    if position in notes:
                        if notes[position]=="DEFINITE":
                            agent.definite=True
                        elif notes[position]=="INDEFINITE":
                            agent.definite=False
                    else:
                        notes[head]="DEFINITE"
                    tokens.append(lemma)
                elif role=="obj":
                    notes[position]="PATIENT" # for now
                    agent=referent(surface)
                    if position in notes:
                        if notes[position]=="DEFINITE":
                            agent.definite=True
                        elif notes[position]=="INDEFINITE":
                            agent.definite=False
                    else:
                        notes[head]="DEFINITE"
                    tokens.append(lemma)
                else:
                    tokens.append(lemma)
        else:
            b+=1
            if gameon:
                pass #print("dofdgfile(): strange line: ",fdgline)
#        print(a," ",u," ",w," ",b)
        try:
            fdgline=fdgfile.readline()
        except:
            print("dofdgfile(): broken line at line "+str(a))
            a += 1
            fdgline=fdgfile.readline()
    print(a," lines seen; ",u," utterances seen ",w," words seen",b," items disregarded")
    fdgfile.close()
    return utterances

In [12]:
us=dofdgfile(fdgtextfile)
for u in us:
    print(u.tokens)
    

dofdgfile(): broken line at line 17317
83058  lines seen;  5727  utterances seen  54057  words seen 19650  items disregarded
['alien', 'treatment', 'be', 'still', 'big', 'mystery']
['n', 'it', 'old', 'trick', 'in', 'rule', 'book', 'there', 'accused', 'full', 'and', 'frank', 'confession', 'down', 'stair', 'but', '1', 'new', 'drama', 'spin', 'good', 'cop/bad', 'cop', 'routine', 'handcuff', 'on', 'tabletop', 'till', 'both', 'just', 'blur']
['hywel', 'bennett', 'be', 'grasping', 'and', 'unscrupulous', 'assistant', 'chief', 'constable', 'desmond', 'iles', 'while', 'aneirin', 'hughes', 'be', 'cid', 'chief', 'colin', 'harpur', 'force', 'to', 'work', 'same', 'case', 'of', 'kidnapping']
['on', 'novel', 'protection', 'bill', 'james', 'run', 'through', 'cardiff', 'underworld', 'try', 'to', 'recover', 'young', 'son', 'of', 'underworld', 'boss', 'swipe', 'as', 'rival', 'gang', 'battle', 'over', 'protection', 'racket']
['it', 'be', 'chance', 'to', 'find', 'son', 'and', 'nail', 'father', 'but', 'inve

['three', 'day', 'on', 'hogmanay', 'night', 'light', 'show', 'normally', 'at', 'home', 'of', 'civil', 'engineer', 'peter', 'smart', 'sheepburn', 'road', 'uddingston']
['new', 'year', 'morning', 'he', 'garage', 'door', 'open', 'and', 'he', 'car', 'go']
['but', 'with', 'holiday', 'period', 'it', 'be', 'not', 'until', 'january', '6', 'that', 'alarm', 'raise']
['peter', 'smart', 'and', 'he', 'wife', 'doris', 'find', 'in', 'bed', 'shoot', 'at', 'through', 'head', 'mrs', 'smart', 'nightdress', 'open']
['next', 'bedroom', 'they', 'michael', 'also', 'lie', 'dead', 'again', 'bullet']
['killer', 'time', 'at', 'house', 'over', 'new', 'year', 'to', 'feed', 'cat', 'it', 'seem']
['take', 'car', 'peter', 'manuel', 'also', 'have', 'audacity', 'to', 'give', 'lift', 'to', 'policeman', 'who', 'be', 'about', 'to', 'join', 'search', 'for', 'isabelle', 'cooke', 'he', 'off', 'he', 'they', 'search', 'in', 'wrong', 'place']
['policeman', 'pause', 'to', 'wonder']
['public', 'alarm', 'reach', 'fever', 'pitch']
[

In [4]:
def dowordspacefile(wordspacefile):
    linepattern = re.compile(r'^\(\"([\-\w]+)\"\s+#S2000([\d\,\-\.\-\+;e]+)*:\s+#S2000([\d\-\+;\.e\,]+)*:\s+#S2000([\.\d\-\+;e\,]+)*:\s+(\d+)\)')
    vecpattern = re.compile(r';?(\d+)([\+\-][\,\.\d]+(e-\d\d)?)')
    alicefile=open(wordspacefile,'r')
    global vectorspace
    vectorspace={}
    global globalfrequency
    globalfrequency={}
    global indexspace
    indexspace={}
    global associationspace
    associationspace={}
    aliceline = alicefile.readline()
    a=1
    b=0
    w=0
    while aliceline:
        a = a+1
        m=linepattern.match(aliceline)
        if m:
            token=m.groups()[0]
            ind=m.groups()[1]
            doc=m.groups()[2]
            ctx=m.groups()[3]
            freq=int(m.groups()[4])
            try:
                sparsevec={}
                cells = re.findall(vecpattern,ctx)
                for c in cells:
                    val = c[1]
                    kop=val.translate(str.maketrans(",",".","+"))
                    try:
                        sparsevec[c[0]] = float(kop)
                    except:
                        print("weird number: ",kop)
                vectorspace[token] = sparsevec
                sparsevec={}
                cells = re.findall(vecpattern,ind)
                for c in cells:
                    val = c[1]
                    kop=val.translate(str.maketrans(",",".","+"))
                    try:
                        sparsevec[c[0]] = float(kop)
                    except:
                        print("weird number: ",kop)
                indexspace[token] = sparsevec
                globalfrequency[token] = freq
                sparsevec={}
                w=w+1
            except:
                print("fail: ",token," ",a)
        else:
            b=b+1
        aliceline = alicefile.readline()
    print(a," lines seen; ",w," items added ",b," items disregarded")
    alicefile.close()

In [5]:
dowordspacefile(wordspacefile)

fail:  --------------------------------------------------------------------------------   2
fail:  smoothgoing   5
fail:  829-7391   167
fail:  23-dec   219
fail:  888pts   255
fail:  30-feb   340
fail:  21-dec   450
fail:  tutich   484
fail:  26-dec   577
fail:  848-0113   607
fail:  898-7108   712
fail:  27-dec   717
fail:  938-9135   741
fail:  938-1133   758
fail:  949-4321   796
fail:  24-dec   829
fail:  848-4158   851
fail:  839-3359   862
fail:  pfu-bleauuuuugggggh   906
fail:  968-9100   916
fail:  25-dec   931
fail:  28-dec   1203
fail:  0723140909070809   1220
fail:  0722140909070809   1227
fail:  29-dec   1294
fail:  259-0333   1684
fail:  259-2337   1685
fail:  288-2132   1704
fail:  90048-4185   1716
fail:  348-2169   1732
fail:  329-2366   1746
fail:  308-2155   1808
fail:  338-9144   1897
fail:  breheny   2156
fail:  379-8309   2241
fail:  0718140909070809   2272
fail:  yassou   2588
fail:  wahooo   2614
fail:  0720130909070809   2809
fail:  399-4393   2818
fail:  349-4

fail:  01761-471771   34917
fail:  whatissit   34979
fail:  0417151612   35174
fail:  269-0633   35348
fail:  288-1434   35366
fail:  249-3600   35577
fail:  299-3600   35599
fail:  369-2627   35702
fail:  239-7662   35710
fail:  278-9457   35814
fail:  65360-0027   35867
fail:  278-5444   35901
fail:  248-6441   35905
fail:  30-sept   36169
fail:  pandol   36282
fail:  sportscope   36552
fail:  415-441-7100   36580
fail:  499-7692   36746
fail:  718-0433   36847
fail:  34742-2007   36852
fail:  688-1477   36876
fail:  798-9426   36941
fail:  autozone   37087
fail:  688-2445   37157
fail:  789-3600   37160
fail:  509-2665   37301
fail:  548-6450   37405
fail:  499-0610   37465
fail:  469-6605   37493
fail:  638-2485   37514
fail:  498-0400   37536
fail:  559-8633   37612
fail:  439-3672   37613
fail:  598-4433   37633
fail:  559-1607   37807
fail:  mullner   37839
fail:  488-6445   37867
fail:  arrrrgggg   38229
fail:  800508   38421
fail:  93-762   38423
fail:  calendar-watching   387

fail:  streller   70116
fail:  dividends5   70245
fail:  anti-virtuoso   70442
fail:  321-aids   70685
fail:  slam-o   70746
fail:  19b2992   71019
fail:  21-oct   71244
fail:  20-oct   71372
fail:  969-0891   71569
fail:  25-oct   71763
fail:  24-oct   71904
fail:  27-oct   72056
fail:  clashpoint   72151
fail:  noooooooo   72161
fail:  26-oct   72185
fail:  011-32-50-370587   72867
fail:  imagescan   72885
fail:  lipshitz   73037
fail:  4296-5627   73116
fail:  datacard   73181
fail:  259-7800   73259
fail:  349-4801   73599
fail:  708-249-1776   73647
fail:  513-281-4700   73691
fail:  zhirinosvky   73839
fail:  why-always-in-my-back-yard   73925
fail:  229-5880   73946
fail:  brennen   74123
fail:  769-3866   74436
fail:  629-8860   74750
fail:  649-3853   74845
fail:  659-4853   74860
fail:  dateland   74921
fail:  759-0808   74977
fail:  799-1805   75009
fail:  afs-info   75267
fail:  mattress-to-go   75326
fail:  629-8888   75331
fail:  499-4805   75364
fail:  armineh   75535
fa

fail:  762-5437   110424
fail:  771-2233   110429
fail:  761-2233   110433
fail:  650-0060   110506
fail:  654-3864   110508
fail:  622-8464   110519
fail:  682-9463   110542
fail:  782-4424   110559
fail:  664-7856   110588
fail:  754-2816   110597
fail:  683-1651   110645
fail:  582-2490   110652
fail:  713-1601   110715
fail:  723-0606   110730
fail:  784-1806   110744
fail:  692-4445   110748
fail:  20410-3000   110784
fail:  562-4462   110844
fail:  534-2864   110867
fail:  570-1051   110950
fail:  433-6611   110953
fail:  legalgard   111135
fail:  561-3223   111178
fail:  492-0465   111207
fail:  482-4464   111214
fail:  434-8851   111256
fail:  solvay   111278
fail:  581-1212   111287
fail:  721-1289   111352
fail:  493-9643   111374
fail:  khadrawys   111389
fail:  1090137   111422
fail:  494990   111855
fail:  310-325   112198
fail:  023127   112394
fail:  202599   112583
fail:  154300   112631
fail:  4348748   112757
fail:  93-0730   113204
fail:  10-1115   113246
fail:  93-0

fail:  650-5577   128944
fail:  673-3377   128947
fail:  766-4926   128991
fail:  720-1522   129008
fail:  biskupic   129020
fail:  733-6318   129082
fail:  660-8557   129085
fail:  793-7316   129110
fail:  436-9931   129206
fail:  chatworth   129225
fail:  011-64-6-835-0022   129238
fail:  526-9967   129255
fail:  496-0922   129290
fail:  643-5396   129333
fail:  433-4315   129337
fail:  656-1986   129373
fail:  441-5703   129382
fail:  553-1333   129447
fail:  596-1935   129472
fail:  483-5373   129480
fail:  453-3361   129506
fail:  496-1960   129543
fail:  563-2313   129581
fail:  450-6556   129582
fail:  533-0319   129588
fail:  533-2311   129593
fail:  466-9955   129598
fail:  726-7998   129599
fail:  782-1191   129608
fail:  522-9100   129646
fail:  763-3383   129647
fail:  2360-g   130094
fail:  306229   130416
fail:  107632   130509
fail:  103648   130572
fail:  131611   130759
fail:  344200   130824
fail:  161650   130886
fail:  274043   130935
fail:  355200   130938
fail:  a

...

fail:  071-405-2519   280293
fail:  818-894-5742   280639
fail:  805-987-0196   280665
fail:  grocholski   281330
fail:  ghermann   281899
fail:  cerebro   281917
fail:  troublegum   281936
fail:  neo-modern   282063
fail:  800-285-5525   282948
fail:  mcdonie   283004
fail:  800-323-4900   283090
fail:  916-322-1394   283239
fail:  011-46-8-16-05-15   283247
fail:  fandel   283433
fail:  beallo   283528
fail:  ophus   283780
fail:  dolinsky   283930
fail:  politics-sociology   284002
fail:  kyotsuke   284238
fail:  flammini   284258
fail:  ashla   284378
fail:  800-634-7711   284589
fail:  chines   284908
fail:  800-777-0112   285012
fail:  95-32255   285119
fail:  schambeck   285296
fail:  boales   285784
fail:  oicvfmnx   285972
fail:  011-61-7124-1900   286009
fail:  lowelljoint   286751
fail:  b10-11   287001
fail:  rifa   288311
fail:  letters12   288832
fail:  irm   289683
fail:  isx   289872
fail:  nixa   290360
fail:  fifty-two-year-old   290614
fail:  lucsko   290780
fail:  e

fail:  0782-326412   361179
fail:  jacksel   361312
fail:  padesky   361384
fail:  98291-0322   361489
fail:  cyberval   361767
fail:  213-250-5600   362048
fail:  labason   362145
fail:  ochone   362173
fail:  moskun   362231
fail:  hahaha   362389
fail:  213-387-2800   362494
fail:  smackle   362544
fail:  8003   363231
fail:  3004   363274
fail:  6014   363558
fail:  6013   363587
fail:  5015   363589
fail:  021   363692
fail:  9020   363727
fail:  3027   363749
fail:  nnnnmmm   363841
fail:  039   363911
fail:  coooool   363914
fail:  213-413-4042   364021
fail:  818-508-9600   364096
fail:  04c   364169
fail:  065   364444
fail:  8064   364471
fail:  6066   364516
fail:  9075   364666
fail:  4079   364675
fail:  07c   364725
fail:  213-740-0647   364763
fail:  213-740-0641   364766
fail:  eccactly   364789
fail:  6080   364857
fail:  09c   365026
fail:  r-retired   365234
fail:  hampsey   365412
fail:  98020   365661
fail:  99026   365668
fail:  89005   365674
fail:  94028   36567

fail:  politics6   401382
fail:  ayrer   401414
fail:  bue-kowski   401658
fail:  karpe   401661
fail:  underdown   401952
fail:  balvir   402115
fail:  borik   402497
fail:  tch-tch   403375
fail:  213-462-5111   403470
fail:  9317   403574
fail:  31c   403646
fail:  31o   403655
fail:  7339   404125
fail:  7346   404390
fail:  9358   404590
fail:  3357   404601
fail:  dogzin   404790
fail:  henderlong   404991
fail:  310-545-2735   405027
fail:  brrrr   405054
fail:  euro-honour   405364
fail:  6387   405441
fail:  sight-seaing   405858
fail:  trashjoint   406103
fail:  jarin   406215
fail:  grrrr   406309
fail:  derva   406655
fail:  80301   406757
fail:  80306   406765
fail:  80308   406769
fail:  86302   406777
fail:  9-329   406782
fail:  98335   406808
fail:  95330   406819
fail:  90302   406867
fail:  80322   406868
fail:  92307   406870
fail:  81321   406873
fail:  94301   406882
fail:  92315   406939
fail:  98362   406979
fail:  95361   406992
fail:  borrrrring   407088
fail:

fail:  6-610   447773
fail:  asclosesasthis   447950
fail:  16dn   448113
fail:  svfm   448283
fail:  svfx   448290
fail:  louisiania   448510
fail:  kalmonsohn   449817
fail:  misterios   450138
fail:  818-989-4377   450158
fail:  0782-292121   450798
fail:  dividen   450849
fail:  207th   451291
fail:  stoner-jive-hip-bop   451333
fail:  0736-62009   451423
fail:  hi-ooooh   451518
fail:  brrrrr   451823
fail:  0782-205124   451917
fail:  roucheville   451930
fail:  0782-204141   452347
fail:  011-43-7245-2501   452481
fail:  pfffffffftt   452865
fail:  r-eally   452943
fail:  shaefer   453058
fail:  bedrms   453203
fail:  robsham   453315
fail:  06170813110907   453704
fail:  leyner   454145
fail:  thhaaaaaannnnnks   454172
fail:  givan   454243
fail:  youvegottanaff   454701
fail:  teacher-coach   455107
fail:  -730   455225
fail:  74c   455422
fail:  303-726-5514   455472
fail:  ooohhh   456023
fail:  78c   456338
fail:  dummkoph   456341
fail:  0796   456488
fail:  linnig   45668

fail:  eeeeeeyyyooooowwww   512180
fail:  asheboro   512285
fail:  kilogrammer   512717
fail:  e-verybody   512799
fail:  tuberhead   513162
fail:  229-story   513635
fail:  california-o   514026
fail:  ad-2525   514281
fail:  tsuneishi   514323
fail:  c-r-r-inge   514592
fail:  tuesday-nov   514668
fail:  mondays-saturdays   514738
fail:  werfelmann   514949
fail:  213-897-3656   515602
fail:  kylius   515695
fail:  kormann   515851
fail:  whaaahooooo   516019
fail:  0782-599667   516450
fail:  utica-rome   516960
fail:  whitney-on-wye   517277
fail:  jaloproctitis   517545
fail:  roaaaar   517659
fail:  okihiro   517663
fail:  pogrand   517746
fail:  312-280-6000   518278
fail:  delauri   518302
fail:  hooooo   519164
fail:  312-464-4818   519255
fail:  geoquiz   519651
fail:  aaaahhhhhh   519998
fail:  800-468-2825   520166
fail:  odiase   520219
fail:  fillman   520298
fail:  800-548-2812   520419
fail:  wineman   520743
fail:  dellaertmoorpark   521693
fail:  shee-veh   521847
fai

In [9]:
def sparseadd(onevec,othvec,weight=1):
    result={}
    try:
        for l in onevec:
            result[l] = onevec[l]
        for k in othvec:
            if k in result:
                result[k] = result[k]+othvec[k]*float(weight)
            else:
                result[k] = othvec[k]*float(weight)
    except:
        print("sparseadd(): error")
    return result
def sparsemultiply(onevec,othvec,weight=1):
    result={}
    try:
        for l in onevec:
            if l in othvec:
                result[l] = onevec[l]*othvec[l]*float(weight)
    except:
        print("sparsemultiply(): error ")
    return result
def sparsexor(onevec,othvec):
    result={}
    try:
        for l in range(len(onevec)):
            if ((l in onevec) and not (l in othvec)):
                result[l] = 1
            if (not (l in onevec) and (l in othvec)):
                result[l] = 1        
    except:
        print("sparsexor(): error ")
    return result

def newrandomvector(n,k):
    vec = {}
    if (k > 0):# no need to be careful about this, right? and k % 2 == 0):
        nonzeros = random.sample(list(range(n)),k)
        negatives = random.sample(nonzeros,k//2)
        for i in nonzeros:
            vec[str(i)] = 1;
        for i in negatives:
            vec[str(i)] = -1;
    return vec

def newoperator(n):
    k = n//10
    return newrandomvector(n,k)

def sparsecosine(xvec,yvec,truncate=True):
    x2 = 0;
    y2 = 0;
    xy = 0;
    try:
        for i in xvec:
            x2 = x2+xvec[i]*xvec[i]
    except:
        print("sparsecosine(): error at position ",i)
    try:
        for j in yvec:
            y2 = y2+yvec[j]*yvec[j]
            if j in xvec:
                xy = xy+xvec[j]*yvec[j]
    except:
        print("sparsecosine(): errors at position ",j)
    if (x2*y2 == 0):
        cos = 0
    else:
        cos = xy/(math.sqrt(x2)*math.sqrt(y2))
    if (truncate):
        cos=int(cos*10000)/10000
    return cos

def sparselength(vec,truncate=True):
    x2 = 0
    length=0
    try:
        for i in vec:
            x2 = x2+vec[i]*vec[i]
    except:
        print("sparselength(): error at position ",i)
    if (x2 > 0):
        length = math.sqrt(x2)
    if (truncate):
        length=int(length*10000)/10000
    return length

def comb(vec,k=0.1):
    newvector={}
    n=int(k*dimensionality/2)
    sorted_items=sorted(vec.items(), key=lambda x:x[1])
    bot=sorted_items[:n]
    top=sorted_items[-n:]
    for l in bot:
        newvector[l[0]]=l[1]
    for l in top:
        newvector[l[0]]=l[1]
    return newvector

def sparsesum(vec):
    s=0
    for i in vec:
        s += float(vec[i])
    return s

def normalise(vec,truncate=True):
    newvector={}
    vlen=sparselength(vec,False)
    if (vlen > 0):
        for i in vec:
            newvector[i]=vec[i]/math.sqrt(vlen*vlen)
    else:
        newvector=vec
    return newvector

def modify(vec,factor):
    newvector={}
    for i in vec:
        if (random.random() > factor):
            newvector[i]=vec[i]
        else:
            newvector[i]=float(vec[i])*(0.5-random.random())*2.0
    return newvector

def createpermutation(k):
    permutation=random.sample(range(k), k)
    return permutation
    
def permute(vector,permutation):
    newvector={}
    try:
        for i in range(len(permutation)):
            if str(i) in vector:
                newvector[str(permutation[i])]=vector[str(i)]
    except:
        newvector=vector
        print("permute(): no permutation done, something wrong")
    return newvector
    

In [10]:
class clause:
    cleanuppattern=re.compile(r'[\.\'\!\*\?\+,;\:\-\/]+')
    def __init__(self,string):
        self.surfacestring=string
        self.cleanedutterance=re.sub(clause.cleanuppattern,"",string)
        self.tokens=self.cleanedutterance.lower().split()
        self.agent=None
        self.event=None
        self.patient=None
        self.instrument=None
        self.location=None
    def __str__(self):
        return "hello"
    
class referent:
    def __init__(self, string):
        self.surfacestring=string
        self.definite=True
        self.number=1
        
class event:
    def __init__(self, string):
        self.surfacestring=string    
        self.tense=None #past present or future
        self.aspect=None #ongoing perfect or pointwise
        self.mood=None #indicative, irreal, potential, optative, not, imperative

def semanticparse(string):
    utterance=clause(string)  
    words=utterance.tokens
    utterance.agent=referent(words.pop(0))       # english grammar: first word is subject
    utterance.agent.definite=True            # for now, assume subject is definite
    mainv=words.pop(0)              # main verb is second word of utterance in english
    if (mainv=="will"):             #    sometimes an auxiliary, for now "will" is accepted
        utterance.event=event(words.pop(0))
        utterance.event.tense="FUTURE"
    else:
        utterance.event=event(mainv)
    det=words.pop(0)                # object carries a determiner 
    utterance.patient=referent(words.pop(0))
    if (det=="a"):                   # if det is "a", object is indefinite
        utterance.patient.definite=False
    else: 
        utterance.patient.definite=True       # else ("the" or "my" or some such) object is definite
    return utterance

# i feed my cat
# i will feed my dog

In [ ]:
def parseutterances(utterances):
    for u in utterances:
        print(u)
        struct=semanticparse(u)
        print("agent\tverb\tpatient")
        if (struct.patient.definite):
            patdef="(def)"
        else:
            patdef="(indef)"
        print(struct.agent.surfacestring,"\t",struct.verb,"\t",struct.patient.surfacestring,patdef)

In [ ]:



utterances=["I feed a cat.", "I feed the cat.", "I feed my cat.", "I fed my cat.", "I saw the dog.", "You will feed the cat.", "I sail a boat.", "You sail my yacht.", "I sailed my boat.", "You will sail the boat."]

parseutterances(utterances)


In [15]:
dimensionality=2000

definite=createpermutation(dimensionality)
indefinite=createpermutation(dimensionality)
future=createpermutation(dimensionality)
past=createpermutation(dimensionality)
present=createpermutation(dimensionality)

#definiteoperator=newoperator(dimensionality)
#indefiniteoperator=newoperator(dimensionality)

agentoperator=newoperator(dimensionality)
patientoperator=newoperator(dimensionality)
mainverboperator=newoperator(dimensionality)

def utterancevector(utterance,lexical=True,morphology=True,syntacticroles=True,morphologyrole=False):
    baremorphology=morphology
    combinedmorphology=lexical
    uttvec={}
    if lexical:
        if (utterance.patient):
            uttvec=sparseadd(uttvec,normalise(comb(vectorspace[utterance.patient.surfacestring])))#,sparselength(vectorspace[utterance.patient]))
        if (utterance.event):
            uttvec=sparseadd(uttvec,normalise(comb(vectorspace[utterance.event.surfacestring])))#,sparselength(vectorspace[utterance.verb]))
        if (utterance.agent):
            uttvec=sparseadd(uttvec,normalise(comb(vectorspace[utterance.agent.surfacestring])))#,sparselength(vectorspace[utterance.agent]))
    if morphology:
        if (utterance.agent):
            if (utterance.agent.definite):
                uttvec=sparseadd(uttvec,normalise(permute(indexspace[utterance.agent.surfacestring],definite)))
            else:
                uttvec=sparseadd(uttvec,normalise(permute(indexspace[utterance.agent.surfacestring],indefinite)))
        if (utterance.patient):
            if (utterance.patient.definite):
                uttvec=sparseadd(uttvec,normalise(permute(indexspace[utterance.patient.surfacestring],definite)))
            else:
     #       uttvec=sparseadd(uttvec,normalise(permute(comb(vectorspace[utterance.patient.surfacestring]),indefinite)))  
                 uttvec=sparseadd(uttvec,normalise(permute(indexspace[utterance.patient.surfacestring],indefinite)))
        if (utterance.event):
            if (utterance.event.tense=="FUTURE"):
                uttvec=sparseadd(uttvec,normalise(permute(vectorspace[utterance.event.surfacestring],future)))
            else:
                uttvec=sparseadd(uttvec,normalise(permute(vectorspace[utterance.event.surfacestring],present)) )  
    if morphologyrole:
        if (utterance.agent):
            if (utterance.agent.definite):
                uttvec=sparseadd(uttvec,normalise(permute(agentoperator,definite)))
            else:
                uttvec=sparseadd(uttvec,normalise(permute(agentoperator,indefinite)))
        if (utterance.patient):
            if (utterance.patient.definite):
                uttvec=sparseadd(uttvec,normalise(permute(patientoperator,definite)))
            else:
     #       uttvec=sparseadd(uttvec,normalise(permute(comb(vectorspace[utterance.patient.surfacestring]),indefinite)))  
                 uttvec=sparseadd(uttvec,normalise(permute(patientoperator,indefinite)))
        if (utterance.event):
            if (utterance.event.tense=="FUTURE"):
                uttvec=sparseadd(uttvec,normalise(permute(mainverboperator,future)))
            else:
                uttvec=sparseadd(uttvec,normalise(permute(mainverboperator,present)) )  
    if syntacticroles:
        if (utterance.patient):
            uttvec=sparseadd(uttvec,sparsemultiply(patientoperator,normalise(indexspace[utterance.patient.surfacestring])))
        if (utterance.event):
            uttvec=sparseadd(uttvec,sparsemultiply(mainverboperator,normalise(indexspace[utterance.event.surfacestring])))
        if (utterance.agent):
            uttvec=sparseadd(uttvec,sparsemultiply(agentoperator,normalise(indexspace[utterance.agent.surfacestring])))
    return uttvec



In [18]:
u=random.choice(us)
print(u.tokens)
v1=utterancevector(u)
u=random.choice(us)
print(u.tokens)
v2=utterancevectordebug(v1,u)
#print(sparsecosine(v1,v2))


['one', 'of', 'great', 'factor', 'be', 'isolation', 'imprison', 'at', 'home', 'and', 'need', 'for', 'adult', 'contact', 'and', 'sympathy']
['court', 'proceeding', 'in', 'february', 'mr', 'west', 'make', 'one', 'brief', 'formal', 'appearance', 'on', 'january', '10', 'but', 'full', 'trial', 'on', 'mass', 'murder', 'allegation', 'not', 'expect', 'until', 'end', 'of', 'year']
0.0
0.0266
0.0104
0.0104
0.0104


#### def utterancevectordebug(v1,utterance,lexical=True,morphology=True,syntacticroles=True):
    baremorphology=morphology
    combinedmorphology=lexical
    uttvec={}
    print(sparsecosine(v1,uttvec))
    if lexical:
        if (utterance.patient):
            uttvec=sparseadd(uttvec,normalise(comb(vectorspace[utterance.patient.surfacestring])))#,sparselength(vectorspace[utterance.patient]))
        if (utterance.event):
            uttvec=sparseadd(uttvec,normalise(comb(vectorspace[utterance.event.surfacestring])))#,sparselength(vectorspace[utterance.verb]))
        if (utterance.agent):
            uttvec=sparseadd(uttvec,normalise(comb(vectorspace[utterance.agent.surfacestring])))#,sparselength(vectorspace[utterance.agent]))
    print(sparsecosine(v1,uttvec))
    if morphology:
        if (utterance.agent):
            if (utterance.agent.definite):
                uttvec=sparseadd(uttvec,normalise(permute(indexspace[utterance.agent.surfacestring],definite)))
            else:
                uttvec=sparseadd(uttvec,normalise(permute(indexspace[utterance.agent.surfacestring],indefinite)))
        if (utterance.patient):
            if (utterance.patient.definite):
                uttvec=sparseadd(uttvec,normalise(permute(indexspace[utterance.patient.surfacestring],definite)))
            else:
     #       uttvec=sparseadd(uttvec,normalise(permute(comb(vectorspace[utterance.patient.surfacestring]),indefinite)))  
                 uttvec=sparseadd(uttvec,normalise(permute(indexspace[utterance.patient.surfacestring],indefinite)))
        if (utterance.event):
            if (utterance.event.tense=="FUTURE"):
                uttvec=sparseadd(uttvec,normalise(permute(vectorspace[utterance.event.surfacestring],future)))
            else:
                uttvec=sparseadd(uttvec,normalise(permute(vectorspace[utterance.event.surfacestring],present)) )  
    print(sparsecosine(v1,uttvec))
    if False:
        if (utterance.agent):
            if (utterance.agent.definite):
                uttvec=sparseadd(uttvec,normalise(permute(agentoperator,definite)))
            else:
                uttvec=sparseadd(uttvec,normalise(permute(agentoperator,indefinite)))
        if (utterance.patient):
            if (utterance.patient.definite):
                uttvec=sparseadd(uttvec,normalise(permute(patientoperator,definite)))
            else:
     #       uttvec=sparseadd(uttvec,normalise(permute(comb(vectorspace[utterance.patient.surfacestring]),indefinite)))  
                 uttvec=sparseadd(uttvec,normalise(permute(patientoperator,indefinite)))
        if (utterance.event):
            if (utterance.event.tense=="FUTURE"):
                uttvec=sparseadd(uttvec,normalise(permute(mainverboperator,future)))
            else:
                uttvec=sparseadd(uttvec,normalise(permute(mainverboperator,present)) )  
    print(sparsecosine(v1,uttvec))
    if syntacticroles:
        if (utterance.patient):
            uttvec=sparseadd(uttvec,sparsemultiply(patientoperator,normalise(indexspace[utterance.patient.surfacestring])))
        if (utterance.event):
            uttvec=sparseadd(uttvec,sparsemultiply(mainverboperator,normalise(indexspace[utterance.event.surfacestring])))
        if (utterance.agent):
            uttvec=sparseadd(uttvec,sparsemultiply(agentoperator,normalise(indexspace[utterance.agent.surfacestring])))
    print(sparsecosine(v1,uttvec))
    return uttvec


In [102]:
u=random.choice(us)
print(u.tokens)
v1=vs[u]
u=random.choice(us)
print(u.tokens)
v2=utterancevectordebug(v1,u)
print("--")

['if', 'there', 'be', 'close', 'friend', 'in', 'who', 'they', 'confide', 'they', 'contact', 'national', 'childbirth', 'trust', 'or', 'mama', 'meet', 'a', 'mum', 'association']
['reserve', 'keep', 'for', 'special', 'additional', 'prize', 'and', 'currently', 'this', 'account', 'for', 'about', 'of', 'sale', 'date', 'around', 'million', 'after', 'first', 'six', 'week', 'of', 'draw']
0.0
0.0166
0.0522
0.0522
0.0539
--


In [ ]:
vs={}
nb={}
for u in us:
    vs[u]=utterancevector(u)
for v in vs:
    max=0
    neighbour=None
    for w in vs:
        if v==w:
            continue
        vwc=sparsecosine(vs[v],vs[w])
        if vwc > max:
            max = vwc
            neighbour=w
    print(v.tokens)
    if neighbour:
        print("\t",max,"\t",neighbour.tokens)
        nb[v]=neighbour

['buckingham', 'palace', 'deny', 'report', 'that', 'prince', 'of', 'wales', 'plan', 'to', 'buy', 'harbourside', 'home', 'in', 'sydney']
	 0.5259 	 ['on', 'november', '5', '1974', 'she', 'leave', 'work', 'at', 'to', 'travel', 'home', 'to', 'ombersley', 'road', 'droitwich', 'and', 'disappear']
['helen', 'chadwick', 'study', 'and', 'draw', 'rare', 'alpine', 'plant', 'here', 'in', 'preparation', 'for', 'major', 'swiss', 'show']
	 0.1864 	 ['they', 'well', 'plant', 'in', 'west', 'allotment', 'which', 'be', 'under', 'm8', 'motorway', 'now']
['he', 'that', 'desire', 'for', 'strong', 'action', 'against', 'individual', 'stem', 'primarily', 'from', 'tabloid', 'press', 'and', 'handful', 'of', 'politician', 'be', 'palpable', 'nonsense', 'in', 'face', 'of', 'increasing', 'fear', 'feel', 'by', 'so', 'many', 'of', 'we', 'citizen', 'towards', 'killer']
	 0.2207 	 ['too', 'much', 'immediate', 'result', 'and', 'improvement', 'also']
['i', 'reckon', 'there', 'be', 'other', 'body', 'plant', 'somewhere']
	

['i', 'have', 'slight', 'shoulder', 'injury', 'but', 'there', 'be', 'and', 'i', 'fit', 'to', 'ride', 'tomorrow']
	 0.814 	 ['mr', 'richard', '91', 'of', 'ilkley', 'yorkshire', 'have', '74', 'driving', 'year', 'and', 'stay', 'with', 'same', 'insurance', 'company']
['it', 'immediately', 'undermine', 'by', 'statement', 'that', 'on', 'average', 'murderer', 'walk', 'street', 'within', '12', 'year']
	 0.3433 	 ['murderer', 'much']
sparsecosine(): errors at position  436
['jack', 'in', 'sweeney', 'even', 'tough', 'about', 'flying', 'squad', 'follow', 'by', 'gentle', 'morse', 'buff', 'of', 'complex', 'crossword', 'university', 'atmosphere', 'of', 'oxford']
	 0.1107 	 ['i', 'he', 'as', 'very', 'hardworking', 'mr', 'newman']
['former', 'colleague', 'and', 'employer', 'describe', 'he', 'as', 'solitary', 'obsess', 'with', 'roman', 'catholicism', 'and', 'supposed', 'evil', 'of', 'abortion']
	 0.7162 	 ['snap', 'clinton', 'at', 'weekend', 'harry', 'benson', 'father', 'found', 'calderpark', 'zoo', 'u

sparsecosine(): error at position  504
['sterling', 'bind', 'to', 'in', 'basket', 'of', 'eu', 'currency', 'wrong', 'level', 'and', 'at', 'wrong', 'time']
	 0.5785 	 ['there', 'bind', 'to', 'be', 'internal', 'bickering', 'though', 'it', 'probable', 'contain', 'way', 'messrs', 'blair', 'and', 'brown', 'manage', 'to', 'impose', 'they', 'authority']
['i', 'it', 'be', 'first', 'time', 'i', 'win', 'bonus']
	 0.4944 	 ['bonus', 'as', 'celtic', 'bank']
sparsecosine(): error at position  568
['change', 'coincide', 'with', 'increase', 'in', 'allergy']
	 1.0 	 ['change', 'in', 'holiday', 'habit', 'coincide', 'with', 'increase', 'in', 'complaint', 'about', 'package', 'holiday', 'in', '1994']
['ian', 'bruce', 'government', 'plan', 'to', 'virtually', 'scrap', 'defence', 'ministry', 'equipment', 'quality', 'watchdog', 'department', 'catastrophic', 'effect', 'british', 'industry', 'and', 'cost', 'taxpayer', 'time', 'what', 'treasury', 'to', 'save']
	 0.3665 	 ['she', 'association', 'with', 'mr', 'mcla

['take', 'car', 'peter', 'manuel', 'also', 'have', 'audacity', 'to', 'give', 'lift', 'to', 'policeman', 'who', 'be', 'about', 'to', 'join', 'search', 'for', 'isabelle', 'cooke', 'he', 'off', 'he', 'they', 'search', 'in', 'wrong', 'place']
	 0.71 	 ['mr', 'richard', '91', 'of', 'ilkley', 'yorkshire', 'have', '74', 'driving', 'year', 'and', 'stay', 'with', 'same', 'insurance', 'company']
['fire', 'discover', 'in', 'kitchen', 'area', 'of', 'four', 'season', 'hotel', 'hotel', 'manager', 'bear', 'houseman', 'meshkati', 'to', 'tackle', 'blaze', 'he', 'but', 'have', 'to', 'drag', 'he', 'own', 'safety']
	 0.4972 	 ['mr', 'west', 'who', 'own', 'house', 'of', 'horror', 'cromwell', 'where', 'police', 'unearth', 'nine', 'corpse', 'discover', 'just', 'before', '1pm', 'in', 'he', 'cell', 'in', 'winson', 'green', 'prison', 'in', 'birmingham']
['scotrail', 'face', 'devastating', 'cash', 'crisis', 'which', 'not', 'lead', 'to', 'service', 'reduce', 'job', 'but', 'which', 'badly', 'undermine', 'scotland'

['morse', 'watch', 'by', '16', 'million', 'in', 'uk', 'country']
	 0.5538 	 ['show', 'i', 'not', 'watch', 'if', 'they', 'pay', 'i']
['you', 'not', 'see', 'where', 'painting', 'stop', 'and', 'photography', 'begin']
	 0.5781 	 ['group', 'investment', 'director', 'mark', 'white', 'see', 'uk', 'footsie', 'index', 'as', 'go', 'to', '3500', 'and', 'dow', 'jones', '4000']
['wednesday', 'home', 'game', 'against', 'leeds', 'not', 'postpone', 'they', 'lead', 'be', 'six', 'point']
	 0.7328 	 ['after', 'mallinson', 'off', 'target', 'penalty', 'attempt', 'but', 'flatley', 'pare', 'scot', 'lead', 'to', 'single', 'point', 'successful', 'penalty', 'kick', 'of', 'he', 'own']
['total', 'of', '40', 'road', 'in', 'region', 'severely', 'affected', 'and', '12', 'close', 'for', 'much', 'of', 'day']
	 0.3196 	 ['13', 'm', 'eac', '15', 'j', 'cooper', 'springburn', '22', 'c', 'youngson', 'metro', 'veteran', '32', '40', 'r', 'thomas', 'hbt']
['what', 'follow', 'be', 'you', 'usual', 'bruce', 'willis', 'action', '

In [74]:
from nltk.parse.stanford import StanfordDependencyParser
path_to_stanford_corenlp = '/home/jussi/Desktop/92.python/stanford-corenlp-full/'
parser_jar='stanford-corenlp.jar'
path_to_models = path_to_stanford_corenlp+'models/'
model_jar='stanford-english-corenlp-models.jar'
dependency_parser = StanfordDependencyParser(path_to_jar=path_to_stanford_corenlp+parser_jar, path_to_models_jar=path_to_models+model_jar)
#dep = dependency_parser.raw_parse('The sailing boat ran aground North-East of Lerwick.')

#print(dep)
#for ii in dep:
#    print(i)
#    for ii in i:
#        print("  ",ii)
#        for jjj in ii.nodes:
#            print("-----",ii.nodes[jjj]['word'],"\t",ii.nodes[jjj]['address'],"-",ii.nodes[jjj]['rel'],"->",ii.nodes[jjj]['head'])
#            print("-----",ii.nodes[jjj]['tag'])
#            for kk in ii.nodes[jjj]:
#                print(kk,end=" ")
#            print()

def semanticparsedep(string):
    utterance=clause(string)
    depnodes=dependency_parser.raw_parse(string)
    for deps in depnodes:
        for nodes in deps:
            if node['rel'] == 'nsubj':
                print(node['word']," is subject")

dd=dependency_parser.raw_parse('It hated me because I spoke to it.')
for d in dd:
    print(d)
def whatevss():
    utterance.agent=referent(words.pop(0))       # english grammar: first word is subject
    utterance.agent.definite=True            # for now, assume subject is definite
    mainv=words.pop(0)              # main verb is second word of utterance in english
    if (mainv=="will"):             #    sometimes an auxiliary, for now "will" is accepted
        utterance.event=event(words.pop(0))
        utterance.event.tense="FUTURE"
    else:
        utterance.event=event(mainv)
    det=words.pop(0)                # object carries a determiner 
    utterance.patient=referent(words.pop(0))
    if (det=="a"):                   # if det is "a", object is indefinite
        utterance.patient.definite=False
    else: 
        utterance.patient.definite=True       # else ("the" or "my" or some such) object is definite
    return utterance

            
            

defaultdict(<function DependencyGraph.__init__.<locals>.<lambda> at 0x7fce10d35c80>,
            {0: {'address': 0,
                 'ctag': 'TOP',
                 'deps': defaultdict(<class 'list'>, {'root': [2]}),
                 'feats': None,
                 'head': None,
                 'lemma': None,
                 'rel': None,
                 'tag': 'TOP',
                 'word': None},
             1: {'address': 1,
                 'ctag': 'PRP',
                 'deps': defaultdict(<class 'list'>, {}),
                 'feats': '_',
                 'head': 2,
                 'lemma': '_',
                 'rel': 'nsubj',
                 'tag': 'PRP',
                 'word': 'It'},
             2: {'address': 2,
                 'ctag': 'VBD',
                 'deps': defaultdict(<class 'list'>,
                                     {'advcl': [6],
                                      'dobj': [3],
                                      'nsubj': [1]}),
               

In [27]:
ttt=nltk.word_tokenize("The sailing boat ran aground North-East of Lerwick.")
ttp=nltk.pos_tag(ttt)
print(ttp)

[('The', 'DT'), ('sailing', 'NN'), ('boat', 'NN'), ('ran', 'VBD'), ('aground', 'IN'), ('North-East', 'NNP'), ('of', 'IN'), ('Lerwick', 'NNP'), ('.', '.')]


## UNIT TEST EMBRYO


In [ ]:
dud=newrandomvector(dimensionality,200) #subject

foo=newrandomvector(dimensionality,200) #object
bar=modify(foo,0.5) #another object

hey=newrandomvector(dimensionality,500) #verb
ho=modify(hey,0.5) #another verb

vectorspace["dud"]=dud
vectorspace["foo"]=foo
vectorspace["bar"]=bar
vectorspace["hey"]=hey
vectorspace["ho"]=ho
# to use the above now indexspace vectors need to be added
indexspace["dud"]=vectorspace["dud"]
indexspace["foo"]=vectorspace["foo"]
indexspace["bar"]=vectorspace["bar"]
indexspace["hey"]=vectorspace["hey"]
indexspace["ho"]=vectorspace["ho"]


utterances1=["I saw a cat.","Cat saw a cat.","I saw the cat."] # which is or should be closer to #1, #2 or #3?
utterances2=["I feed a cat.", "I feed the cat.","I saw a cat."] # ditto
utterances3=["dud hey a bar","dud hey a foo","dud ho the bar","dud hey the bar"]

utterances=utterances2
utterancespace1={}
utterancespace2={}
utterancespace3={}
utterancespace4={}
utterancespace5={}
utterancespace6={}
for u in utterances:
    struct=semanticparse(u)
    utterancespace1[u] = utterancevector(struct,True,False,False) #only lex
    utterancespace2[u] = utterancevector(struct,False,True,False) # only morph
    utterancespace3[u] = utterancevector(struct,True,True,False)
    utterancespace4[u] = utterancevector(struct,True,False,True)
    utterancespace5[u] = utterancevector(struct,False,True,True)
    utterancespace6[u] = utterancevector(struct,True,True,True)
for u in utterances:
    print(u)
    for v in utterances:
        print(
            sparsecosine(utterancespace1[u],utterancespace1[v]),"\t",
              sparsecosine(utterancespace2[u],utterancespace2[v]),"\t",
#              sparsecosine(utterancespace3[u],utterancespace3[v]),"\t",
#              sparsecosine(utterancespace4[u],utterancespace4[v]),"\t",
#              sparsecosine(utterancespace5[u],utterancespace5[v]),"\t",
              sparsecosine(utterancespace6[u],utterancespace6[v]),"\t",
              v)
for u in utterances:
    print(u)
    print(
        "Indef",
        sparsecosine(utterancespace2[u],permute(patientoperator,indefinite)),"\t",
        sparsecosine(utterancespace6[u],permute(patientoperator,indefinite)) )
    print(
        "Def",
        sparsecosine(utterancespace2[u],permute(patientoperator,definite)),"\t",
        sparsecosine(utterancespace6[u],permute(patientoperator,definite)) )
        


In [ ]:
c = semanticparse("cat saw a cat")

In [ ]:
c.surfacestring

In [ ]:
c.surfacestring